In [ ]:
import os
import wfdb
import pandas as pd
import numpy as np
from scipy.io import loadmat
import torch
from torch.utils.data import Dataset
from torchvision import transforms
# from biosppy.signals.ecg import correct_rpeaks
from scipy.signal import resample
import sys
from seaborn import scatterplot
sys.path.append('/home/bohdan/PeakDetector')
from modules.baseline import ResNet
# from sklearn.preprocessing import MinMaxScaler

In [ ]:
import torch
from torch import nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
from torch.nn import functional as F
import numpy as np
import time
from sklearn.model_selection import train_test_split
import tqdm
import shutil
import os
import seaborn as sns
import matplotlib.pyplot as plt
import imageio
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
# from utils import normalize
from scipy import signal
import glob


from scipy.signal import find_peaks

def save_checkpoint(state, is_best, checkpoint='checkpoint_fpn', filename='checkpoint.pth.tar', snapshot=None):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)

    if snapshot and state.epoch % snapshot == 0:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'checkpoint_{}.pth.tar'.format(state.epoch)))

    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv1d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv1d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )


class SignalNet(nn.Module):
    def __init__(self, n_class, n_channels):
        super().__init__()

        self.dconv_down1 = double_conv(n_channels, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool1d(2)
        self.upsample = nn.Upsample(scale_factor=2)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv1d(64, n_class, 1)

    def forward(self, x):
        x_shapes = []

        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        x_shapes.append(x.shape)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        x_shapes.append(x.shape)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x_shapes.append(x.shape)

        x = self.dconv_down4(x)
        x_shapes.append(x.shape)

        x = self.upsample(x)
        x_shapes.append(x.shape)

        x = torch.cat([x, conv3], dim=1)
        x_shapes.append(x.shape)

        x = self.dconv_up3(x)
        x_shapes.append(x.shape)

        x = self.upsample(x)
        x_shapes.append(x.shape)

        x = torch.cat([x, conv2], dim=1)
        x_shapes.append(x.shape)

        x = self.dconv_up2(x)
        x_shapes.append(x.shape)

        x = self.upsample(x)
        x_shapes.append(x.shape)

        x = torch.cat([x, conv1], dim=1)
        x_shapes.append(x.shape)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out
    
class IncResBlock(nn.Module):
    def __init__(self, inplanes, planes, convstr=1, convsize=15, convpadding=7):
        super(IncResBlock, self).__init__()
        self.Inputconv1x1 = nn.Conv1d(inplanes, planes, kernel_size=1, stride=1, bias=False)
        self.conv1_1 = nn.Sequential(
            nn.Conv1d(
                in_channels=inplanes,
                out_channels=planes//4,
                kernel_size=convsize,
                stride=convstr,
                padding=convpadding
            ),
            nn.BatchNorm1d(planes//4))
        self.conv1_2 = nn.Sequential(
            nn.Conv1d(
                inplanes,
                planes//4,
                kernel_size=1,
                stride=convstr,
                padding=0,
                bias=False
            ),
            nn.BatchNorm1d(planes//4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(
                in_channels=planes//4,
                out_channels=planes//4,
                kernel_size=convsize + 2,
                stride=convstr,
                padding=convpadding + 1
            ),
            nn.BatchNorm1d(planes//4))
        self.conv1_3 = nn.Sequential(
            nn.Conv1d(
                inplanes,
                planes//4,
                kernel_size=1,
                stride=convstr,
                padding=0,
                bias=False
            ),
            nn.BatchNorm1d(planes//4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(
                in_channels=planes//4,
                out_channels=planes//4,
                kernel_size=convsize + 4,
                stride=convstr,
                padding=convpadding + 2
            ),
            nn.BatchNorm1d(planes//4))
        self.conv1_4 = nn.Sequential(
            nn.Conv1d(inplanes, planes//4, kernel_size=1, stride = convstr, padding=0, bias=False),
            nn.BatchNorm1d(planes//4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(
                in_channels=planes//4,
                out_channels=planes//4,
                kernel_size=convsize + 6,
                stride=convstr,
                padding=convpadding+3
            ),
            nn.BatchNorm1d(planes//4))
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = self.Inputconv1x1(x)

        c1 = self.conv1_1(x)
        c2 = self.conv1_2(x)
        c3 = self.conv1_3(x)
        c4 = self.conv1_4(x)

        out = torch.cat([c1, c2, c3, c4], 1)
        out += residual
        out = self.relu(out)

        return out


class IncUNet (nn.Module):
    def __init__(self, in_shape):
        super(IncUNet, self).__init__()
        in_channels, height, width = in_shape
        self.e1 = nn.Sequential(
            nn.Conv1d(in_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2, inplace=True),
            IncResBlock(64, 64))
        self.e2 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(128),
            IncResBlock(128, 128))
        self.e2add = nn.Sequential(
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128))
        self.e3 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2,),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(256),
            IncResBlock(256, 256))
        self.e4 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(256, 256, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))
        self.e4add = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512))
        self.e5 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.e6 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.e6add = nn.Sequential(
            nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512))

        self.e7 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.e8 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512))

        self.d1 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 512, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.d2 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.d3 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(1024, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.5),
            IncResBlock(512, 512))

        self.d4 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.d5 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.d6 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(1024, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            IncResBlock(512, 512))

        self.d7 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(1024, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(256, 256, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm1d(256),
            IncResBlock(256, 256))

        self.d8 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128))

        self.d9 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128))

        self.d10 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(256, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(64))

        self.out_l = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(128, in_channels, kernel_size=4, stride=2, padding=1))

    def forward(self, x):
        en1 = self.e1(x)
        en2 = self.e2(en1)
        en2add = self.e2add(en2)
        en3 = self.e3(en2add)
        en4 = self.e4(en3)
        en4add = self.e4add(en4)
        en5 = self.e5(en4add)
        en6 = self.e6(en5)
        en6add = self.e6add(en6)
        en7 = self.e7(en6add)
        en8 = self.e8(en7)

        de1_ = self.d1(en8)
        de1 = torch.cat([en7, de1_], 1)
        de2_ = self.d2(de1)
        de2 = torch.cat([en6add, de2_], 1)
        de3_ = self.d3(de2)
        de3 = torch.cat([en6, de3_], 1)
        de4_ = self.d4(de3)
        de4 = torch.cat([en5, de4_], 1)
        de5_ = self.d5(de4)
        de5 = torch.cat([en4add, de5_], 1)
        de6_ = self.d6(de5)
        de6 = torch.cat([en4, de6_], 1)
        de7_ = self.d7(de6)
        de7 = torch.cat([en3, de7_], 1)
        de8_ = self.d8(de7)
        de8 = torch.cat([en2add, de8_], 1)
        de9_ = self.d9(de8)
        de9 = torch.cat([en2, de9_], 1)
        de10_ = self.d10(de9)
        de10 = torch.cat([en1, de10_], 1)
        out = self.out_l(de10)

        return out

In [ ]:
def min_max_normalize(X):
    return (X - np.min(X)) / (np.max(X) - np.min(X))

class SegmentationDataset(Dataset):
    SIGNAL_PATH_COLUMN = 'PathToSignal'
    TARGET_PATH_COLUMN = 'PathToTarget'
    DATASET_COLUMN = 'Dataset'
    CLASS_COLUMN = 'Disease'
    LENGTH_COLUMN = 'Length'

    def __init__(self, root_dir, df,
                 transform=None,
                 augmentation=None,
                 config=None,
                 softmax=False,
                 fs=250):
        """
        :param root_dir:
        :param df:
        :param transform:
        :param augmentation:
        :param config:
        """
        if config is None:
            config = {}
        self.root_dir = root_dir
        self.df = df
        self.df[self.SIGNAL_PATH_COLUMN] = self.df[self.SIGNAL_PATH_COLUMN].apply(str)
        self.df[self.TARGET_PATH_COLUMN] = self.df[self.TARGET_PATH_COLUMN].apply(str)

        self.transform = transform
        self.augmentation = augmentation
        self.config = config
        self.softmax = softmax
        self.fs = fs

        self.signal_cache = {}

    def __len__(self):
        return self.df.shape[0]

    def __load_signal(self, path, start=0, end=-1, lead=None):
        if path not in self.signal_cache.keys():
            if path.endswith('.mat'):
                x = loadmat(path)['ecg'].ravel().ravel()
            else:
                record = wfdb.rdrecord(path)
                record_fs = record.fs

                if lead is None or lead not in record.sig_name:
                    x = record.p_signal[:, 0]
                else:
                    lead_idx = np.where(np.array(record.sig_name) == lead)[0][0]
                    x = record.p_signal[:, lead_idx].ravel()
                x = resample(x, num=int(x.size * self.fs / record_fs))
            x_norm = min_max_normalize(x)
            self.signal_cache[path] = x_norm

        return self.signal_cache[path][start:end]

    def __load_target(self, path, start, end):
        target = np.load(path)['target'][start:end].ravel()

        if self.softmax:
            target = np.array([
                target,
                1. - target
            ])
        return target

    def __getitem__(self, idx):
        info = self.df.iloc[idx]
        start = info.Start
        end = info.End
        if 'Lead' in info.index:
            lead = info.Lead
        else:
            lead = None

        path_to_signal = os.path.join(
            self.root_dir,
            info[self.DATASET_COLUMN],
            info[self.SIGNAL_PATH_COLUMN]
        )

        path_to_target = os.path.join(
            self.root_dir,
            info[self.DATASET_COLUMN],
            info[self.TARGET_PATH_COLUMN]
        )

        loaded_signal = self.__load_signal(path_to_signal, lead=lead, start=start, end=end)
        loaded_target = self.__load_target(path_to_target, start, end)

        x = torch.tensor(
            loaded_signal, dtype=torch.float32
        ).unsqueeze(0)

        y = torch.tensor(
            loaded_target, dtype=torch.float32
        )

        if self.transform:
            x = self.transform(x)

        return x, y

In [ ]:
from scipy.signal import find_peaks
import numpy as np
import biosppy

from torch.nn.functional import pad

# def peaks_finder(Z, threshold=0.6):
#     Z[:20] = 0
#     Z[-20:] = 0

#     return find_peaks(Z, distance=50, height=0.3)[0]#find_peaks(Z, distance=220)[0]

def peaks_finder(Z, threshold=0.6):
    all_peaks_found = False

    X = Z.copy()
    peaks = []

    while not all_peaks_found:
        peak = np.argmax(X)
        if X[peak] > threshold:
            peaks.append(peak)
            trim_from = max(0, peak - 200)
            trim_to = min(len(X), peak + 200)
            X[trim_from:trim_to] = 0
        else:
            all_peaks_found = True

    return sorted(peaks)

def normalize(X):
    return (X - X.min()) / (X.max() - X.min())

def filter_ecg(ecg, sampling_rate=250, filter_type="FIR", filter_band="bandpass",
               filter_frequency=[2, 115], filter_order=2):
    if filter_type in ["FIR", "butter", "cheby1", "cheby2", "ellip", "bessel"]:
        order = int(filter_order * sampling_rate)
        filtered, _, _ = biosppy.tools.filter_signal(signal=ecg,
                                                     ftype=filter_type,
                                                     band=filter_band,
                                                     order=order,
                                                     frequency=filter_frequency,
                                                     sampling_rate=sampling_rate)
    else:
        raise ValueError('You can use only "FIR", "butter", "cheby1", "cheby2", "ellip", "bessel" filter types')

    return filtered

In [ ]:
def create_segmentation_model(config):
    if config['seg_model_name'].lower() == 'rpnet':
        seg_model = IncUNet(in_shape=config['seg_inp_shape'])
    else:
        seg_model = SignalNet(n_channels=config['seg_inp_shape'], n_class=config['seg_num_classes'])
        
    if config['seg_path_to_weights']:
        seg_model.load_state_dict(torch.load(config['seg_path_to_weights'])['state_dict'])
    seg_model.eval()
    return seg_model

def create_classifier(config):
    if config['classifier_model_name'].lower() == 'resnet':
        model = ResNet()
    elif config['classifier_model_name'].lower() == 'inceptiontime':
        model = InceptionTime()
    else:
        model = RNNModel()
    if config['classifier_path_to_weights']:
        model = model.load_from_checkpoint(config['classifier_path_to_weights'])
    
    model.eval()
    return model

class PVCDetector(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        self.seg_threshold = config['seg_threshold']
        self.seg_model = create_segmentation_model(config)
        self.classifier = create_classifier(config)
        
    def forward(self, x):
        
        seg_out = self.seg_model(x)
        
        out = seg_out.cpu().detach().squeeze(0).numpy()
        peaks = np.array([peaks_finder(out[i], threshold=self.seg_threshold) for i in range(out.shape[0])])[0]
        left_pad = 200 - peaks[0] if peaks[0] - 200 < 0 else 0
        right_pad = (peaks[-1] + 200+left_pad) - x.shape[2] if peaks[-1] + 200 > x.shape[2] else 0
        padded_x = pad(x, (left_pad, right_pad), mode='constant', value=0)
        
        output = []
        for peak in peaks:
            peak_ind_with_pad = peak + left_pad
            cur_heart_beat = padded_x[:, :, peak_ind_with_pad-150:peak_ind_with_pad+150]
            classifier_out = self.classifier(cur_heart_beat)
            pvc_class = classifier_out.argmax().item()
            output.append((peak, pvc_class))
        return output, out

In [ ]:
config = {
    # SEGMENTATION
    'seg_model_name': 'unet', # rpnet or unet
    'seg_inp_shape': 1, #(1, 1, 5000),
    'seg_num_classes': 1,
    'seg_threshold': 0.6,
    
#     'seg_path_to_weights': '/home/ikachko/Galene_2/checkpoints/rpnet_2_classes_5000_smooth_l1/rpnet_2_classes_5000_smooth_l1.pt', 
#     'seg_path_to_weights': '/home/ikachko/Galene_2/checkpoints/unet_2_classes_5000_smooth_l1/unet_2_classes_5000_smooth_l1.pt',
    'seg_path_to_weights': '/home/ikachko/Galene_2/checkpoints/mitbih_unet_5000_smooth_l1/by_f1_0.98mitbih_unet_5000_smooth_l1.pt',
    # CLASSIFICATION
    'classifier_model_name': 'ResNet', # LSTM or IncTime or ResNet
    'classifier_inp_shape': 1,
    'classifier_num_classes': 1,
#     'classifier_path_to_weights': '/home/bohdan/PeakDetector/mit_svdb_exps/version_29/checkpoints/epoch=232.ckpt'
    'classifier_path_to_weights': '/home/bohdan/PeakDetector/logs/raw_input_mit_bih_lstm/version_22/checkpoints/epoch=3.ckpt'
}

In [ ]:
import random 
from biosppy.signals.ecg import correct_rpeaks
import math

class ChinaDataset(Dataset):
    SIGNAL_PATH_COLUMN = 'PathToData'
    CLASS_COLUMN = 'Label'

    def __init__(self, root_dir):
        """
        :param root_dir:
        :param df:
        :param transform:
        :param augmentation:
        :param config:
        """
        random.seed(42)
        self.root_dir = root_dir
        self.resample_ratio = 250 / 400
        self.patients = ['A02.mat', 'A03.mat', 'A04.mat', 'A07.mat', 'A08.mat', 'A10.mat']
        self.signal_cache_x = {}
        self.signal_cache_y = {}
    def __len__(self) -> int:
        return len(self.patients)

    def __getitem__(self, idx):

        path_to_file = self.patients[idx]


        if path_to_file not in self.signal_cache_x.keys():
            cur_pat = np.expand_dims(np.squeeze(np.asarray(
                loadmat(os.path.join(self.root_dir, path_to_file))['ecg'], dtype=np.float64), 1), 1)
            cur_pat = normalize(cur_pat)
            self.signal_cache_x[path_to_file] = resample(cur_pat, num=int(cur_pat.size * 250 / 400))
            
            self.signal_cache_y[path_to_file] = np.squeeze(loadmat(os.path.join(self.root_dir, path_to_file.replace('A', 'R')))['ref'][0][0][1], 1)
            self.signal_cache_y[path_to_file] = (self.signal_cache_y[path_to_file] * self.resample_ratio).astype(int)
            self.signal_cache_y[path_to_file] = correct_rpeaks(self.signal_cache_x[path_to_file], self.signal_cache_y[path_to_file], sampling_rate=250)['rpeaks']

        x = self.signal_cache_x[path_to_file]
        x = np.expand_dims(np.squeeze(x, 1), 0)
        x = torch.tensor(x.copy(), dtype=torch.float64)

        y = self.signal_cache_y[path_to_file]

        return x.float(), y


In [ ]:
file_p = '/datasets/extra_space2/ikachko/ecg/china_challenge/TrainingSet'
d = ChinaDataset(file_p)

In [ ]:
x, y = d[0]

In [ ]:
path_to_csv = '/datasets/extra_space2/ikachko/ecg/svdb_conv_25_5000.csv'
root_dir = '/datasets/extra_space2/ikachko/ecg/'
df = pd.read_csv(path_to_csv)

transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = SegmentationDataset(
    root_dir=root_dir,
    df=df
)

In [ ]:
window_size = 5000
start = 800
step = 400

detector = PVCDetector(config)
sigmoid = nn.Sigmoid()
for i in tqdm.tqdm(range(len(d))):
#     x, y = dataset[i]
    x, y = d[i]
    
    cur_peaks = []
    for i in range(math.ceil(len(x[0]) / step)):
        cur_x = x[:, start:start+window_size]
        if len(cur_x[0]) != window_size:
            cur_x = pad(cur_x, (0, window_size - len(cur_x[0])), mode='constant', value=0)
                
        preds, raw_out = detector(cur_x.unsqueeze(0))
        
#         preds = [(x[0]+start, x[1]) for x in preds]
        
        cur_x = cur_x[0].detach()
        pvc_peaks = [x[0] for x in preds if x[1]]
        normal_peaks = [x[0] for x in preds if not x[1]]
        
        plt.figure(figsize=(20,10))
        plt.plot(cur_x)
        plt.plot(raw_out[0])
        plt.scatter(pvc_peaks, cur_x[pvc_peaks], color='red', s=100)
        plt.scatter(normal_peaks, cur_x[normal_peaks], color='green', s=100)
        start += step
        break


# Thrash

In [ ]:
from biosppy.signals.ecg import correct_rpeaks
from scipy import signal
import os
import wfdb
import pandas as pd
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm 
import gc
import itertools

import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import transforms
from torch.nn.functional import pad

from scipy.signal import resample
import sys
from seaborn import scatterplot
import random 
from biosppy.signals.ecg import correct_rpeaks
import math

import matplotlib.pyplot as plt

sys.path.append('/home/bohdan/PeakDetector')

from modules.baseline import ResNet
# from local_models import SignalNet
# from utils import normalize, peaks_finder, filter_ecg


In [ ]:
df_name = '/datasets/extra_space2/ikachko/ecg/PVC_mit_bih.csv'
d = pd.read_csv(df_name)
exp_path = '/datasets/extra_space2/ikachko/ecg/'

x_or = np.asarray(wfdb.rdrecord(os.path.join(exp_path, d['Dataset'].iloc[0], str(d['PathToData'].iloc[0]))).p_signal[:, 0], dtype=np.float64)
x = signal.resample(x, num=int(x.size * 250 / 360))

In [ ]:
a = correct_rpeaks(x, d[d['PathToData']==100]['Location'], sampling_rate=250)['rpeaks']

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(x[:10000])
merged = a #d[d['PathToData']==100]['Location']
fif = [x for x in merged if x < 10000 ]
plt.scatter(fif, x[:10000][fif], color='red')

In [ ]:
pd.concat([pd.read_csv('/home/ikachko/Galene_2/classic_metrics_1.csv'),pd.read_csv('/home/ikachko/Galene_2/classic_metrics_5.csv')])

In [ ]:
def get_num_of_correct_peaks(peaks_pred, peaks_true, eps=25):
    """
    TP -- number of predicted peaks, that are close to true peaks
        If dist(peaks_pred, peaks_true[i]) < eps:
            TP += 1
    FP -- number of peaks, that I found, but it was not true
        len(peaks_pred) - TP
    FN -- number of peaks, that I did not found
        len(peak_true) - TP
    :param peaks_pred:
    :param peaks_true:
    :param eps:
    :return:
    """
    N_all = len(peaks_true)

    TP = 0
    if len(peaks_pred) == 0 or len(peaks_true) == 0:
        TP = 0
        FP = 0
        FN = len(peaks_true) - TP
        N_all = len(peaks_true)
        return TP, FP, FN, N_all
    
    distances = np.min(np.array([abs(peaks_true - peaks_pred[i]) for i in range(len(peaks_pred))]), 0)

    TP = (distances < eps).astype(int).sum()
    FP = len(peaks_pred) - TP
    FN = len(peaks_true) - TP

    return TP, FP, FN, N_all

import random 
from biosppy.signals.ecg import correct_rpeaks
import math

class ChinaDataset(Dataset):
    SIGNAL_PATH_COLUMN = 'PathToData'
    CLASS_COLUMN = 'Label'

    def __init__(self, root_dir):
        """
        :param root_dir:
        :param df:
        :param transform:
        :param augmentation:
        :param config:
        """
        random.seed(42)
        self.root_dir = root_dir
        self.resample_ratio = 250 / 400
        self.patients = ['A02.mat', 'A03.mat', 'A04.mat', 'A07.mat', 'A08.mat', 'A10.mat']
        self.signal_cache_x = {}
        self.signal_cache_y = {}
    def __len__(self) -> int:
        return len(self.patients)

    def __getitem__(self, idx):

        path_to_file = self.patients[idx]


        if path_to_file not in self.signal_cache_x.keys():
            cur_pat = np.expand_dims(np.squeeze(np.asarray(
                loadmat(os.path.join(self.root_dir, path_to_file))['ecg'], dtype=np.float64), 1), 1)
#             cur_pat = normalize(cur_pat)
            self.signal_cache_x[path_to_file] = resample(cur_pat, num=int(cur_pat.size * 250 / 400))
            
            self.signal_cache_y[path_to_file] = np.squeeze(loadmat(os.path.join(self.root_dir, path_to_file.replace('A', 'R')))['ref'][0][0][1], 1)
            self.signal_cache_y[path_to_file] = (self.signal_cache_y[path_to_file] * self.resample_ratio).astype(int)
            self.signal_cache_y[path_to_file] = correct_rpeaks(self.signal_cache_x[path_to_file], self.signal_cache_y[path_to_file], sampling_rate=250)['rpeaks']

        x = self.signal_cache_x[path_to_file]
        x = np.expand_dims(np.squeeze(x, 1), 0)
        x = torch.tensor(x.copy(), dtype=torch.float64)

        y = self.signal_cache_y[path_to_file]

        return x.float(), y


In [ ]:
file_p = '/datasets/extra_space2/ikachko/ecg/china_challenge/TrainingSet'
d = ChinaDataset(file_p)

In [ ]:
for i in tqdm(range(len(d))):
    if d.patients[i] == 'A04.mat':
        continue
    x, y = d[i]
    pred = pd.read_csv('/home/ikachko/Galene_2/classic_peaks_{}.csv'.format(str(i)))
    pred = pred[pred['Algorithm'] == 'Hamilton']['Location']
    iTP, iFP, iFN, N_a = get_num_of_correct_peaks(np.array(pred), y)
    TP = iTP
    FP = iFP
    FN = iFN
    N_true = N_a
    
    acc = TP / len(y)
    print(round(acc, 2))

In [ ]:
pd.value_counts(pd.read_csv('/home/ikachko/Galene_2/classic_peaks_{}.csv'.format(str(0)))['Algorithm'])

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(x[0][:10000])
fif = [x for x in pred if x < 10000]
kok = [x for x in y if x < 10000]
plt.scatter(kok, x[0][:10000][kok], color='green',linewidths=10)
plt.scatter(fif, x[0][:10000][fif], color='red')


In [ ]:
df = pd.read_csv('/datasets/extra_space2/ikachko/ecg/mitdb_svdb_resampled.csv')

In [ ]:
d = df

In [ ]:
x_or = np.asarray(wfdb.rdrecord(os.path.join(exp_path, d['Dataset'].iloc[0], str(d['PathToData'].iloc[0]))).p_signal[:, 0], dtype=np.float64)

In [ ]:
plt.plot(x_or[:1000])

In [ ]:
df